In [2]:
from models.CVAE import CVAE
from experiments import MNISTGenExperiment
from lightning.pytorch import Trainer
from lightning.pytorch.utilities.seed import seed_everything
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger
from pathlib import Path
import os



DATA_PATH = os.path.join(os.getcwd(),"data\\MNIST_data")
OUTPUT_DIR = os.path.join(os.path.join(os.path.join(os.getcwd(), "runs"), "MNISTGen"), "logs")
NAME = "CVAE"
MNIST_CLASSES_NUM = 10
MNIST_IMG_SHP = (1, 28, 28)
latent_length = 8



for i, kl_weight in enumerate([0.0001, 0.0005, 0.001, 0.01]):
    seed_everything(1234, True)

    vae_model = CVAE(MNIST_CLASSES_NUM, MNIST_IMG_SHP, latent_length, kl_weight=kl_weight)
    experiment = MNISTGenExperiment.MNISTGenExperiment(vae_model)

    data = MNISTGenExperiment.get_data(DATA_PATH)

    Path(OUTPUT_DIR).mkdir(exist_ok=True, parents=True)
    tb_logger = TensorBoardLogger(save_dir=OUTPUT_DIR, name=NAME, version=i)
    csv_logger = CSVLogger(save_dir=tb_logger.log_dir, name='csv_file')

    runner = Trainer(logger=[tb_logger, csv_logger],
                    callbacks=[
                        #  LearningRateMonitor(),
                        ModelCheckpoint(save_top_k=2, 
                                        dirpath =os.path.join(tb_logger.log_dir , "checkpoints"), 
                                        monitor= "val_loss",
                                        save_last= True),
                    ],
                    max_epochs=20)


    runner.fit(experiment, datamodule=data)


c:\Users\lotan\AppData\Local\Programs\Python\Python37\lib\site-packages\lightning\pytorch\utilities\seed.py:48: LightningDeprecationWarning: `lightning.pytorch.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning.fabric.utilities.seed.seed_everything` instead.
  "`lightning.pytorch.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be"
Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type | Params
-------------------------------
0 | model | CVAE | 821 K 
-------------------------------
821 K     Trainable params
0         Non-trainable params
821 K     Total params
3.286     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Display 5 samples from test


AttributeError: 'tuple' object has no attribute 'shape'

Train MNIST CVAE models for different KL_Weights